In [1]:
%pip install -q langchain langchain-community python-dotenv chromadb sentence-transformers
%pip install pypdf -q
%pip install tf-keras

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: C:\Users\Admin\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [2]:
import sys
sys.path.append('RAG_TECHNIQUES')

import os
import json
from typing import List, Dict, Any
from dotenv import load_dotenv
from langchain_community.chat_models import ChatOllama
from langchain.chains import RetrievalQA
from pydantic import BaseModel, Field

In [3]:
# Definition of helper_functions content
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import SentenceTransformerEmbeddings
from langchain_community.vectorstores import Chroma

def read_pdf_to_string(file_path: str) -> str:
    """Reads a PDF file and returns its content as a single string."""
    loader = PyPDFLoader(file_path)
    docs = loader.load()
    return "".join([doc.page_content for doc in docs])

def encode_from_string(text_content: str) -> Chroma:
    """Encodes text content and creates a Chroma vector store."""
    embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
    # Split text content into chunks to avoid exceeding embedding model limits or context window
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.create_documents([text_content])
    db = Chroma.from_documents(texts, embeddings)
    return db

print("helper_functions defined")

helper_functions defined


In [4]:
load_dotenv()
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE" #Intels Library
os.makedirs('data', exist_ok=True)
print("Setup complete")

Setup complete


In [5]:
llm = ChatOllama(
    model="llama3.2:1b",
    temperature=0,
)

print("Llama 3.2 initialized!")

Llama 3.2 initialized!


C:\Users\Admin\AppData\Local\Temp\ipykernel_13476\3148274586.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(


In [6]:
import os

def select_and_process_pdf():
    """
    Returns the fixed path to the Cyclone Emission Control PDF.
    """
    print("=" * 80)
    print("USING DEFAULT PDF FILE")
    print("=" * 80)

    # Define your fixed path here
    pdf_path = os.path.abspath("data/Cyclone_Emission_Control.pdf")

    if not os.path.exists(pdf_path):
        print(f"⚠️ File not found at: {pdf_path}")
    else:
        print(f"✓ Using PDF: {pdf_path}")

    return pdf_path

print("PDF selection function ready!")


PDF selection function ready!


In [7]:
def initialize_rag_system(pdf_path: str = None):
    """
    Initialize the RAG system with a specified or selected PDF.

    Args:
        pdf_path: Optional path to PDF. If None, user will be prompted to upload.

    Returns:
        tuple: (qa_chain, retriever, content, pdf_path)
    """
    # Get PDF path
    if pdf_path is None:
        pdf_path = select_and_process_pdf()

    print(f"\n[Processing] Reading PDF: {pdf_path}")

    # Load and process document
    content = read_pdf_to_string(pdf_path)
    print(f"✓ PDF loaded: {len(content)} characters")

    print("[Processing] Creating vector embeddings...")
    vectorstore = encode_from_string(content)
    retriever = vectorstore.as_retriever(search_kwargs={"k": 4})
    print("✓ Vector store created")

    # Create QA chain
    print("[Processing] Initializing QA chain...")
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        return_source_documents=True
    )
    print("✓ RAG system ready!")
    print("=" * 80)

    return qa_chain, retriever, content, pdf_path

print("Initialization function ready!")

Initialization function ready!


In [8]:
def get_user_feedback(query: str, response: str, relevance: int, quality: int, comments: str = "") -> Dict[str, Any]:
    """
    Create a feedback entry.

    Args:
        query: The user's query
        response: The system's response
        relevance: Relevance score (1-5)
        quality: Quality score (1-5)
        comments: Optional comments
    """
    return {
        "query": query,
        "response": response,
        "relevance": int(relevance),
        "quality": int(quality),
        "comments": comments
    }

def store_feedback(feedback: Dict[str, Any], filepath: str = "data/feedback_data.json"):
    """Store feedback to JSON file."""
    os.makedirs(os.path.dirname(filepath), exist_ok=True)

    # Read existing data
    existing_data = []
    if os.path.exists(filepath):
        try:
            with open(filepath, "r") as f:
                content = f.read().strip()
                if content:
                    # Handle both line-delimited and array formats
                    if content.startswith('['):
                        existing_data = json.loads(content)
                    else:
                        for line in content.split('\n'):
                            if line.strip():
                                existing_data.append(json.loads(line))
        except json.JSONDecodeError:
            print("Warning: Could not parse existing feedback file. Starting fresh.")
            existing_data = []

    # Append new feedback
    existing_data.append(feedback)

    # Write back as proper JSON array
    with open(filepath, "w") as f:
        json.dump(existing_data, f, indent=2)

    print(f"✓ Feedback stored successfully. Total feedback entries: {len(existing_data)}")

def load_feedback_data(filepath: str = "data/feedback_data.json") -> List[Dict[str, Any]]:
    """Load feedback data from JSON file."""
    if not os.path.exists(filepath):
        print("No feedback data file found. Starting with empty feedback.")
        return []

    try:
        with open(filepath, "r") as f:
            content = f.read().strip()
            if not content:
                return []

            # Handle both line-delimited and array formats
            if content.startswith('['):
                return json.loads(content)
            else:
                feedback_data = []
                for line in content.split('\n'):
                    if line.strip():
                        feedback_data.append(json.loads(line))
                return feedback_data
    except json.JSONDecodeError as e:
        print(f"Error loading feedback data: {e}")
        return []

print("Feedback management functions ready!")

Feedback management functions ready!


In [9]:
def check_relevance_with_llm(query: str, feedback_query: str, doc_content: str, feedback_response: str, llm) -> bool:
    """
    Use LLM to check if feedback is relevant to current query and document.

    Returns:
        bool: True if relevant, False otherwise
    """
    prompt = f"""You are an expert at determining relevance between queries and documents.

Current Query: {query}
Feedback Query: {feedback_query}
Document Content (first 500 chars): {doc_content[:500]}
Feedback Response (first 300 chars): {feedback_response[:300]}

Task: Determine if the feedback response is relevant to BOTH:
1. The current query topic
2. The document content

Consider semantic similarity, not just exact word matches.

Respond with ONLY 'Yes' or 'No'.
Answer:"""

    try:
        response = llm.invoke(prompt)
        answer_text = response.content.strip().lower() if hasattr(response, 'content') else str(response).strip().lower()
        return 'yes' in answer_text
    except Exception as e:
        print(f"Error in relevance checking: {e}")
        return False

def adjust_relevance_scores(query: str, docs: List[Any], feedback_data: List[Dict[str, Any]], llm) -> List[Any]:
    """
    Adjust document relevance scores based on historical feedback.

    Args:
        query: Current user query
        docs: Retrieved documents
        feedback_data: Historical feedback data
        llm: Language model for relevance checking

    Returns:
        List of documents sorted by adjusted relevance scores
    """
    if not feedback_data:
        print("No feedback data available for adjustment.")
        return docs

    print(f"Adjusting relevance scores using {len(feedback_data)} feedback entries...")

    for doc in docs:
        # Initialize relevance score if not present
        if 'relevance_score' not in doc.metadata:
            doc.metadata['relevance_score'] = 1.0

        relevant_feedback_count = 0
        total_relevance = 0

        for feedback in feedback_data:
            # Use LLM to check if this feedback is relevant
            is_relevant = check_relevance_with_llm(
                query=query,
                feedback_query=feedback['query'],
                doc_content=doc.page_content,
                feedback_response=feedback['response'],
                llm=llm
            )

            if is_relevant:
                relevant_feedback_count += 1
                total_relevance += feedback['relevance']

        # Adjust score based on relevant feedback
        if relevant_feedback_count > 0:
            avg_relevance = total_relevance / relevant_feedback_count
            # Normalize to -1 to 1 scale (3 is neutral on 1-5 scale)
            adjustment_factor = (avg_relevance - 3) / 2
            doc.metadata['relevance_score'] *= (1 + adjustment_factor * 0.3)  # 30% max adjustment
            print(f"  Document adjusted: {relevant_feedback_count} relevant feedback, avg score: {avg_relevance:.2f}")

    # Sort by adjusted scores
    sorted_docs = sorted(docs, key=lambda x: x.metadata.get('relevance_score', 1.0), reverse=True)
    return sorted_docs

print("Relevance adjustment functions ready!")

Relevance adjustment functions ready!


In [10]:
def fine_tune_index(feedback_data: List[Dict[str, Any]], original_content: str) -> Any:
    """
    Create an enhanced index incorporating high-quality feedback.

    Args:
        feedback_data: Historical feedback data
        original_content: Original document content

    Returns:
        New vectorstore with enhanced content
    """
    if not feedback_data:
        print("No feedback data available for fine-tuning.")
        return encode_from_string(original_content)

    # Filter high-quality responses (relevance >= 4 AND quality >= 4)
    good_responses = [
        f for f in feedback_data
        if f.get('relevance', 0) >= 4 and f.get('quality', 0) >= 4
    ]

    print(f"Fine-tuning index with {len(good_responses)} high-quality responses...")

    if not good_responses:
        print("No high-quality responses found. Using original content only.")
        return encode_from_string(original_content)

    # Create additional context from high-quality Q&A pairs
    additional_texts = []
    for f in good_responses:
        # Format as Q&A pair for better semantic matching
        qa_text = f"Question: {f['query']}\nAnswer: {f['response']}"
        additional_texts.append(qa_text)

    # Combine all texts
    combined_content = original_content + "\n\n" + "\n\n".join(additional_texts)

    # Create new vectorstore
    new_vectorstore = encode_from_string(combined_content)
    print(f"✓ Index fine-tuned with {len(additional_texts)} additional Q&A pairs")

    return new_vectorstore

print(" Index fine-tuning function ready!")

 Index fine-tuning function ready!


In [11]:
qa_chain, retriever, content, current_pdf = initialize_rag_system()

print(f"\n System initialized with: {current_pdf}")

USING DEFAULT PDF FILE
✓ Using PDF: c:\Users\Admin\Desktop\New folder\data\Cyclone_Emission_Control.pdf

[Processing] Reading PDF: c:\Users\Admin\Desktop\New folder\data\Cyclone_Emission_Control.pdf
✓ PDF loaded: 24644 characters
[Processing] Creating vector embeddings...


C:\Users\Admin\AppData\Local\Temp\ipykernel_13476\4130052877.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = SentenceTransformerEmbeddings(model_name="all-MiniLM-L6-v2")
C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sentence_transformers\cross_encoder\CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


✓ Vector store created
[Processing] Initializing QA chain...
✓ RAG system ready!

 System initialized with: c:\Users\Admin\Desktop\New folder\data\Cyclone_Emission_Control.pdf


In [18]:
query = " What is the intended use of the Cyclone Separator according to the manual?"
print(f"Query: {query}\n")

# Get response
print("Getting response...")
result = qa_chain.invoke({"query": query})
response = result["result"]

print("\n Answer:")
print(response)

# Store response for feedback
current_query = query
current_response = response

Query:  What is the intended use of the Cyclone Separator according to the manual?

Getting response...

 Answer:
According to the manual, the Cyclone Separator is "exclusively intended for the removal of solid particles contained in fumes given off by heat generators."


In [19]:
relevance = 5  
quality = 4    
comments = "Good explanation but could be more detailed"
# Feedback Store
feedback = get_user_feedback(current_query, current_response, relevance, quality, comments)
store_feedback(feedback)

print(f"\nFeedback recorded:")
print(f"  Relevance: {relevance}/5")
print(f"  Quality: {quality}/5")
print(f"  Comments: {comments}")

✓ Feedback stored successfully. Total feedback entries: 2

Feedback recorded:
  Relevance: 5/5
  Quality: 4/5
  Comments: Good explanation but could be more detailed


In [20]:
print("APPLYING FEEDBACK TO IMPROVE SYSTEM")

all_feedback = load_feedback_data()
print(f"\nTotal feedback entries: {len(all_feedback)}")

print("\nAdjusting relevance scores...")
docs = retriever.get_relevant_documents(current_query)
adjusted_docs = adjust_relevance_scores(current_query, docs, all_feedback, llm)

print(f"\nTop document (adjusted):")
print(f"  Content preview: {adjusted_docs[0].page_content[:150]}...")
print(f"  Relevance score: {adjusted_docs[0].metadata.get('relevance_score', 'N/A')}")

print("\nFine-tuning index with high-quality feedback...")
new_vectorstore = fine_tune_index(all_feedback, content)
retriever = new_vectorstore.as_retriever(search_kwargs={"k": 4})

# Create new QA chain with updated retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)

print("\n System updated with feedback!")

APPLYING FEEDBACK TO IMPROVE SYSTEM

Total feedback entries: 2

Adjusting relevance scores...
Adjusting relevance scores using 2 feedback entries...

Top document (adjusted):
  Content preview: Question:  What is the intended use of the Cyclone Separator according to the manual?
Answer: According to the manual, the Cyclone Separator is "exclu...
  Relevance score: 1.0

Fine-tuning index with high-quality feedback...
Fine-tuning index with 2 high-quality responses...


C:\Users\Admin\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\transformers\tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


✓ Index fine-tuned with 2 additional Q&A pairs

 System updated with feedback!


In [21]:
query2 = "Explain the principle of operation of the Cyclone Separator"
print(f"\nQuery: {query2}\n")

print(" Getting response with updated system...")
result2 = qa_chain.invoke({"query": query2})
response2 = result2["result"]

print("\n Answer:")
print(response2)

# Store for feedback
current_query = query2
current_response = response2


Query: Explain the principle of operation of the Cyclone Separator

 Getting response with updated system...

 Answer:
I don't know. The manual doesn't mention a specific principle of operation for the Cyclone Separator. It only states that it is "exclusively intended for the removal of solid particles contained in fumes given off by heat generators."


In [22]:

feedback_data = load_feedback_data()

print("FEEDBACK STATISTICS")
print(f"\nTotal feedback entries: {len(feedback_data)}")

if feedback_data:
    avg_relevance = sum(f.get('relevance', 0) for f in feedback_data) / len(feedback_data)
    avg_quality = sum(f.get('quality', 0) for f in feedback_data) / len(feedback_data)
    print(f"Average relevance score: {avg_relevance:.2f}/5")
    print(f"Average quality score: {avg_quality:.2f}/5")

    high_quality = len([f for f in feedback_data if f.get('relevance', 0) >= 4 and f.get('quality', 0) >= 4])
    print(f"High-quality responses: {high_quality}")

    print("\nAll feedback entries:")
    for i, f in enumerate(feedback_data, 1):
        print(f"\n{i}. Query: {f['query']}")
        print(f"   Relevance: {f['relevance']}/5, Quality: {f['quality']}/5")
        if f.get('comments'):
            print(f"   Comments: {f['comments']}")

FEEDBACK STATISTICS

Total feedback entries: 2
Average relevance score: 5.00/5
Average quality score: 4.00/5
High-quality responses: 2

All feedback entries:

1. Query:  What is the intended use of the Cyclone Separator according to the manual?
   Relevance: 5/5, Quality: 4/5
   Comments: Good explanation but could be more detailed

2. Query:  What is the intended use of the Cyclone Separator according to the manual?
   Relevance: 5/5, Quality: 4/5
   Comments: Good explanation but could be more detailed


In [23]:
my_query = "What are the consequences of using the Cyclone Separator for improper purposes?"  # CHANGE THIS
print(f"Query: {my_query}\n")

print(" Getting response...")
my_result = qa_chain.invoke({"query": my_query})
my_response = my_result["result"]

print("\n Answer:")
print(my_response)

# Store for feedback
current_query = my_query
current_response = my_response


Query: What are the consequences of using the Cyclone Separator for improper purposes?

 Getting response...

 Answer:
According to the text, there are no specific consequences mentioned for using the Cyclone Separator for improper purposes. It only states that "Guarantee of manufacturer liability cannot be called upon in case of damages to people or objects due to: a. improper Cyclone Separator use;" which implies that improper use may lead to liability, but it does not specify what those consequences are.
